In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [7]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Preprocess the Data

In [8]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Encode Categorical Variables

In [9]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

Onehot Encoder 'Geography'

In [10]:
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
# concatenate onehot encoded col with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


Split the data into features and target

In [16]:
X=data.drop('EstimatedSalary',axis=1)
Y=data['EstimatedSalary']
X,Y

(      CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
 0             619       0   42       2       0.00              1          1   
 1             608       0   41       1   83807.86              1          0   
 2             502       0   42       8  159660.80              3          1   
 3             699       0   39       1       0.00              2          0   
 4             850       0   43       2  125510.82              1          1   
 ...           ...     ...  ...     ...        ...            ...        ...   
 9995          771       1   39       5       0.00              2          1   
 9996          516       1   35      10   57369.61              1          1   
 9997          709       0   36       7       0.00              1          0   
 9998          772       1   42       3   75075.31              2          1   
 9999          792       0   28       4  130142.79              1          1   
 
       IsActiveMember  Exited  Geograp

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=3)

In [15]:
# Scale the feature data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_train

array([[ 0.0862834 , -1.09499335, -0.6601382 , ...,  1.00125078,
        -0.58023704, -0.57542576],
       [ 0.31430986,  0.91324755, -0.75549256, ..., -0.99875078,
         1.72343359, -0.57542576],
       [-1.82084701, -1.09499335,  0.57946842, ...,  1.00125078,
        -0.58023704, -0.57542576],
       ...,
       [ 0.28321534,  0.91324755,  1.53301197, ..., -0.99875078,
         1.72343359, -0.57542576],
       [-0.51487727, -1.09499335,  0.198051  , ...,  1.00125078,
        -0.58023704, -0.57542576],
       [-0.7325389 ,  0.91324755, -0.37407514, ..., -0.99875078,
         1.72343359, -0.57542576]])

In [17]:
with open('label_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

---
ANN Regression Problem Statement

In [18]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
# Building the model
model=Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1
  Dense(32,activation='relu'), #HL2
  Dense(1) #output layer
])
# compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

c:\Users\ROHIT GUPTA\Desktop\GenAI-Course-Materials-Krish_Naik\ChurnANN_Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Setting up the Logs

In [29]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

# setting up the tensorboard
log_dir='regression_logs/fit'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [21]:
# Setting up the early stopping callback
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

Train the Model

In [30]:
history=model.fit(
  X_train,Y_train,
  validation_data=(X_test,Y_test),
  epochs=100,
  callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 49971.7891 - mae: 49971.7891 - val_loss: 49990.8984 - val_mae: 49990.8984
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 49507.9922 - mae: 49507.9922 - val_loss: 50018.1094 - val_mae: 50018.1094
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 49619.2461 - mae: 49619.2461 - val_loss: 50009.1367 - val_mae: 50009.1367
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 49808.3281 - mae: 49808.3281 - val_loss: 50031.0273 - val_mae: 50031.0273
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 48947.8789 - mae: 48947.8789 - val_loss: 50009.5664 - val_mae: 50009.5664
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 50165.8555 - mae: 50165.8555 - val_loss: 49996.3945 - val_mae: 49996.3945
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 49241.7656 - mae: 49241.7656 - val_loss: 50016.3516 - val_mae: 50016.3516
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - 

In [31]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:
%tensorboard --logdir regression_logs/fit20250208-193335

Reusing TensorBoard on port 6007 (pid 4200), started 0:01:07 ago. (Use '!kill 4200' to kill it.)

Evaluate the model on the test data

In [35]:
test_loss,test_mae=model.evaluate(X_test,Y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 50519.1875 - mae: 50519.1875
Test MAE: 49990.8984375


In [37]:
model.save('regression_model.h5')